In [3]:
ROOT_DIR ='/data/user/home/mhossai5/DPP-New2025'
!git clone https://github.com/brendaferrari/AutoPaDELPy.git -l {ROOT_DIR}/utils

fatal: destination path '/data/user/home/mhossai5/DPP-New2025/utils' already exists and is not an empty directory.


In [4]:
import os
import sys
sys.path.insert(0,os.path.join(
    ROOT_DIR,'utils'
))
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
df_final = pd.read_csv(os.path.join(
    ROOT_DIR,'data/dpp4-26-03-25.csv'
))
df_final.head()

,id,smiles,standard_value,standard_type,standard_units,target_organism,target_pref_name,pIC50,source,label,target,splits
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.663540,ChEMBL,inactive,1,train
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.387216,ChEMBL,inactive,1,train
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.823909,ChEMBL,inactive,1,train
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.301030,ChEMBL,inactive,1,train
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.725842,ChEMBL,inactive,1,train


In [6]:
# PaDEL Descriptors

In [7]:
from functions.fingerprint_functions import FingerprintFunctions

In [9]:
if not os.path.exists(os.path.join(ROOT_DIR,'data/CDKextended-dpp4.csv')):
    os.chdir(os.path.join(ROOT_DIR,'utils'))
    smi_path =  'dataset.smi'
    df_final[['smiles','id']].to_csv(
       smi_path, sep='\t', index=False, header=False,
    )
    FingerprintFunctions().do_CDKextended(smi_path)
    os.rename(
        os.path.join(ROOT_DIR,'utils/CDKextended.csv'),
        os.path.join(ROOT_DIR,'data/CDKextended-dpp4.csv'),
    )
    os.remove(smi_path)
    os.chdir(os.path.join(ROOT_DIR,'notebook'))

In [10]:
df_final['CDKextended'] = pd.read_csv( os.path.join(ROOT_DIR,'data/CDKextended-dpp4.csv')).drop('Name',axis = 1).values.tolist()

In [11]:
# Morgan 

In [12]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np
from rdkit import rdBase
rdBase.DisableLog('rdApp.warning')

In [13]:
def smiles_to_fp(smiles, radius=2, num_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return list(np.zeros(num_bits))
    return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, num_bits))

In [14]:
df_final['ECFP'] = df_final['smiles'].progress_apply(lambda x: smiles_to_fp(x))
df_final['ECFP_2048'] = df_final['smiles'].progress_apply(lambda x: smiles_to_fp(x,num_bits = 2048))
df_final['ECFP_512'] = df_final['smiles'].progress_apply(lambda x: smiles_to_fp(x,num_bits = 512))

100%|██████████| 6563/6563 [00:04<00:00, 1354.11it/s]


In [15]:
df_final.head()

,id,smiles,standard_value,standard_type,standard_units,target_organism,target_pref_name,pIC50,source,label,target,splits,CDKextended,ECFP,ECFP_2048,ECFP_512
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.663540,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.387216,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.823909,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.301030,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.725842,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [16]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

In [17]:
# Load Llama model and tokenizer
model_name = "meta-llama/Llama-3.2-1B"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
tokenizer.pad_token = tokenizer.eos_token
# Function to extract embeddings
def smiles_to_embedding(smiles):
    """Convert SMILES string to Llama embeddings."""
    inputs = tokenizer(smiles, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy().squeeze()  # Mean pooling

    return embedding

df_final['Llama3-2_Embedding'] = df_final['smiles'].progress_apply(lambda x: smiles_to_embedding(x))

# Print sample embeddings
df_final.head()

2025-03-28 22:13:37.164540: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-28 22:13:37.164670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-28 22:13:37.407088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 22:13:37.955510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-28 22:13:51.123782: W tensorflow/compiler/tf2

,id,smiles,standard_value,standard_type,standard_units,target_organism,target_pref_name,pIC50,source,label,target,splits,CDKextended,ECFP,ECFP_2048,ECFP_512,Llama3-2_Embedding
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.663540,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.49902532, 2.9190297, -0.23703773, -0.25883..."
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.387216,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.9239326, 2.7885501, 0.38945127, 0.3505689,..."
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.823909,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.41079238, 2.760039, 0.025078857, -0.101544..."
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.301030,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.13337657, 2.8452492, 0.630767, 0.08860778,..."
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.725842,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6948919, 2.6412396, 0.3524785, -0.21108356..."


In [18]:
# Load ChemBERT2 model and tokenizer
model_name = "seyonec/ChemBERTa_zinc250k_v2_40k"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# Function to extract embeddings
def smiles_to_embedding(smiles):
    """Convert SMILES string to ChemBERT2 embeddings."""
    inputs = tokenizer(smiles, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy().squeeze()  # Mean pooling

    return embedding

df_final['ChemBERT2_Embedding'] = df_final['smiles'].progress_apply(lambda x: smiles_to_embedding(x))

# Print sample embeddings
df_final.head()

100%|██████████| 6563/6563 [00:25<00:00, 259.91it/s]


,id,smiles,standard_value,standard_type,standard_units,target_organism,target_pref_name,pIC50,source,label,target,splits,CDKextended,ECFP,ECFP_2048,ECFP_512,Llama3-2_Embedding,ChemBERT2_Embedding
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.663540,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.49902532, 2.9190297, -0.23703773, -0.25883...","[-0.38957188, -0.6216704, 0.4703551, 0.1626513..."
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.387216,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.9239326, 2.7885501, 0.38945127, 0.3505689,...","[-0.3062373, -0.24003226, 0.14207578, 0.525902..."
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.823909,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.41079238, 2.760039, 0.025078857, -0.101544...","[-0.3257358, -0.012994111, 0.17750713, -0.0522..."
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.301030,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.13337657, 2.8452492, 0.630767, 0.08860778,...","[-0.0054997415, -0.37361592, 0.015422794, 0.30..."
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.725842,ChEMBL,inactive,1,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6948919, 2.6412396, 0.3524785, -0.21108356...","[-0.6008385, -0.16555355, 0.1292091, 0.9035775..."


In [24]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski

 # Function to compute physicochemical properties
def compute_physicochemical_properties(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {
            "MolWt": np.nan, "LogP": np.nan, "TPSA": np.nan,
            "HBD": np.nan, "HBA": np.nan, "RotatableBonds": np.nan
        }
    
    return {
        "MolWt": Descriptors.MolWt(mol),               # Molecular Weight
        "LogP": Crippen.MolLogP(mol),                  # LogP (Hydrophobicity)
        "TPSA": Descriptors.TPSA(mol),                 # Topological Polar Surface Area
        "HBD": Lipinski.NumHDonors(mol),               # Hydrogen Bond Donors
        "HBA": Lipinski.NumHAcceptors(mol),            # Hydrogen Bond Acceptors
        "RotatableBonds": Lipinski.NumRotatableBonds(mol)  # Rotatable Bonds
    }

# Apply the function and expand results into columns
df_final['Physiochemical'] = df_final['smiles'].apply(lambda x: np.nan_to_num(list(compute_physicochemical_properties(x).values())))

[22:43:53] Explicit valence for atom # 30 Ga, 4, is greater than permitted


In [26]:
df_final.to_parquet(os.path.join(
    ROOT_DIR,'data/dpp4-26-03-25-feat.parquet'
),index = False)

In [25]:
df_final.head()

,id,smiles,standard_value,standard_type,standard_units,target_organism,target_pref_name,pIC50,source,label,...,ECFP_512,Llama3-2_Embedding,ChemBERT2_Embedding,MolWt,LogP,TPSA,HBD,HBA,RotatableBonds,Physiochemical
0,CHEMBL93558,N[C@@H](CC1CCCCC1)C(=O)N1CCCCC1,217000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.663540,ChEMBL,inactive,...,"[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.49902532, 2.9190297, -0.23703773, -0.25883...","[-0.38957188, -0.6216704, 0.4703551, 0.1626513...",238.375,2.2966,46.33,1.0,2.0,3.0,"[238.37499999999994, 2.2966000000000006, 46.33..."
1,CHEMBL443622,C[C@H](N)C(=O)N1CCCC1,41000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.387216,ChEMBL,inactive,...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.9239326, 2.7885501, 0.38945127, 0.3505689,...","[-0.3062373, -0.24003226, 0.14207578, 0.525902...",142.202,-0.0440,46.33,1.0,2.0,1.0,"[142.20199999999997, -0.04400000000000004, 46...."
2,CHEMBL403882,O=C([C@@H]1CCCN1)N1CCCC1,15000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,4.823909,ChEMBL,inactive,...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.41079238, 2.760039, 0.025078857, -0.101544...","[-0.3257358, -0.012994111, 0.17750713, -0.0522...",168.240,0.3608,32.34,1.0,2.0,1.0,"[168.23999999999998, 0.3608, 32.34, 1.0, 2.0, ..."
3,CHEMBL328655,S=C(C1CCCN1)N1CCCC1,500000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.301030,ChEMBL,inactive,...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.13337657, 2.8452492, 0.630767, 0.08860778,...","[-0.0054997415, -0.37361592, 0.015422794, 0.30...",184.308,1.1616,15.27,1.0,2.0,1.0,"[184.30799999999996, 1.1616, 15.27, 1.0, 2.0, ..."
4,CHEMBL328795,NC(=O)CC(N)C(=O)N1CCCC1,188000.0,IC50,nM,Homo sapiens,Dipeptidyl peptidase IV,3.725842,ChEMBL,inactive,...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6948919, 2.6412396, 0.3524785, -0.21108356...","[-0.6008385, -0.16555355, 0.1292091, 0.9035775...",185.227,-1.1885,89.42,2.0,3.0,3.0,"[185.22699999999995, -1.1885000000000001, 89.4..."


In [23]:
df_final['ChemBERT2_Embedding'][0].shape

(768,)